In [2]:
from aylienapiclient import textapi
c = textapi.Client("f40063af", "ddd790cf8730e5934f0a416f4ac44b2a")
s = c.Sentiment({'text': 'John is a very good football player!'})

In [3]:
s

{'polarity': 'positive',
 'polarity_confidence': 0.9940106272697449,
 'subjectivity': 'objective',
 'subjectivity_confidence': 0.943706847162803,
 'text': 'John is a very good football player!'}

In [96]:
#!/usr/bin/python
import psycopg2
import sys
import pprint
from datetime import datetime
import re
import pandas.io.sql as sql
import pandas as pd
import csv
import nltk

def get_entities():
    conn_string = "host='localhost' dbname='newsapp' user='postgres' password='postgres'"
    print ("Connecting to database\n    ->%s" % (conn_string))
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    df_table = sql.read_sql("SELECT backend_article.uniqueid, backend_article.publishedat, backend_article.url, backend_article.source, backend_sentiment.score, backend_article.text FROM backend_article INNER JOIN backend_sentiment ON backend_article.uniqueid = backend_sentiment.article;", conn)  
    df_table["publishedat"] = [d.to_pydatetime().date() for d in df_table["publishedat"]]
    df_table["publishedat"] =  [d.strftime('%d_%m_%Y') if not pd.isnull(d) else '' for d in df_table["publishedat"]]

    df_table["length"] = [len(text) for text in df_table["text"]]
   
    print(len(df_table))
    return df_table

df_table = get_entities()
df_table_date = df_table[df_table.publishedat == '06_07_2017']
df_table_date = df_table_date[:50]

from aylienapiclient import textapi
c = textapi.Client("f40063af", "ddd790cf8730e5934f0a416f4ac44b2a")
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

from nltk import tokenize
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sid = SIA()

df_table_date['aylien_polarity'] = 0
df_table_date['cleaned_text'] = 0
df_table_date['nltk_polarity'] = 0
df_table_date['resp'] = 0

for index, row in df_table_date.iterrows():
    df_table_date.loc[index, 'cleaned_text'] = re.sub('\n','', row['text']) 

for index, row in df_table_date.iterrows():
    aresp = c.Sentiment({'text': row['cleaned_text']})
    resp_val = aresp['polarity']
    df_table_date.loc[index, 'aylien_polarity'] = resp_val
    lines_list = tokenizer.tokenize(row['cleaned_text'])
    for sentence in lines_list:
        ss = sid.polarity_scores(sentence)
    df_table_date.loc[index, 'nltk_polarity'] = ss['compound']
    
df_short =  df_table_date.loc[:,['uniqueid','url','score','aylien_polarity', 'nltk_polarity']]
df_short.to_csv('df_short.csv')









Connecting to database
    ->host='localhost' dbname='newsapp' user='postgres' password='postgres'
5773


In [91]:

df_short =  df_table_date.loc[:,['uniqueid','url','score','aylien_polarity']]
df_short.to_csv('df_short.csv')


In [90]:
df_short

,uniqueid,url,score,aylien_polarity
174,2205608177,http://www.bbc.co.uk/news/science-environment-...,0.162573,neutral
205,280689826,https://www.economist.com/news/books-and-arts/...,-0.501039,neutral
209,3583391377,https://www.economist.com/news/leaders/2172481...,-0.177860,negative
309,1685282208,https://www.theguardian.com/society/2017/jul/0...,-0.161113,negative
321,1831373204,https://www.theguardian.com/politics/2017/jul/...,0.796804,neutral
384,65036243,http://www.telegraph.co.uk/news/2017/07/05/one...,-0.210309,neutral
466,4051102392,http://www.aljazeera.com/news/2017/07/ready-fo...,-0.487629,neutral
471,1548602654,http://www.bbc.co.uk/news/world-europe-40515054,-0.526047,negative
487,3492387083,http://www.bbc.co.uk/news/world-africa-40515325,-0.594527,neutral
499,2039599106,http://www.reuters.com/article/us-virginia-sho...,-0.692197,neutral


In [74]:
df_table_date

,uniqueid,publishedat,url,source,score,text,length,aylien_polarity,cleaned_text,resp
174,2205608177,06_07_2017,http://www.bbc.co.uk/news/science-environment-...,bbc-news,0.162573,The European and Japanese satellites that make...,3828,neutral,The European and Japanese satellites that make...,"{'subjectivity': 'subjective', 'polarity_confi..."
205,280689826,06_07_2017,https://www.economist.com/news/books-and-arts/...,the-economist,-0.501039,\nDestined for War: Can America and China Esca...,3788,neutral,Destined for War: Can America and China Escape...,"{'subjectivity': 'subjective', 'polarity_confi..."
209,3583391377,06_07_2017,https://www.economist.com/news/leaders/2172481...,the-economist,-0.177860,WHAT do you call someone who is over 65 but no...,3568,negative,WHAT do you call someone who is over 65 but no...,"{'subjectivity': 'subjective', 'polarity_confi..."
309,1685282208,06_07_2017,https://www.theguardian.com/society/2017/jul/0...,the-guardian-uk,-0.161113,\n\n\n\nOne in three nursing homes has failed ...,6075,negative,One in three nursing homes has failed its offi...,"{'subjectivity': 'subjective', 'polarity_confi..."
321,1831373204,06_07_2017,https://www.theguardian.com/politics/2017/jul/...,the-guardian-uk,0.796804,\n\n\n\nMinisters must work faster to ensure t...,1858,neutral,Ministers must work faster to ensure that afte...,"{'subjectivity': 'subjective', 'polarity_confi..."
